In [ ]:
!pip install git+https://github.com/Total-RD/pymgrid/
!pip install stable-baselines3[extra]

In [ ]:
import time # Necessary to evaluate frugality
from pymgrid.Environments.pymgrid_cspla import MicroGridEnv # Imposed Environment
import numpy as np
import json 
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike

In [ ]:
import pickle

"""
The buildings mentionned below are specific to the hackathon and are not available in this repo.
You can replace them with any MicroGrid object generated from pymgrid
"""

with open('building_1.pkl', 'rb') as f:
    building_1 = pickle.load(f)
    building_1.train_test_split()

with open('building_2.pkl', 'rb') as f:
    building_2 = pickle.load(f)
    building_2.train_test_split()
    
with open('building_3.pkl', 'rb') as f:
    building_3 = pickle.load(f)
    building_3.train_test_split()

buildings = [building_1, building_2, building_3]
env = MicroGridEnv(env_config={'microgrid':building_3})
model = A2C('MlpPolicy', env, verbose=0)

In [ ]:
def rule_based(building):
    building_data = building.get_updated_values()
    total_building_cost = 0
    price_seen = []
    load_per_day = list(np.zeros(7))
    i = 0
    k = True
    
    while building.done == False:
        t = building_data['hour']
        load = building_data['load']
        pv = building_data['pv']
        battery_soc = building_data['battery_soc']
        capa_to_charge = building_data['capa_to_charge']
        capa_to_dischare = building_data['capa_to_discharge']
        grid_price_import = building_data['grid_price_import']
        
        if grid_price_import not in price_seen:
            price_seen.append(grid_price_import)
            

        p_disc = max(0, min(load-pv, capa_to_dischare, building.battery.p_discharge_max))
        p_char = max(0, min(pv-load, capa_to_charge, building.battery.p_charge_max))

        if i < 7*24: #first 7 days


            if load - pv >=  0:
                control_dict = {'battery_charge': 0, #batereas ar vteni
                                    'battery_discharge': p_disc, # batareidan vteni imdenit ramdenic batareashia an tu raodenoba didia imdenit vteni ramdenic mchirdeba
                                    'grid_import': max(0, load-pv-p_disc), #momaq danarcheni ramac bataream da siantlem ver sheavso
                                    'grid_export':0,
                                    'pv_consummed': min(pv, load), #pv_consumed aris pv mxolod
                                    'genset': 0
                               }


            if load - pv <  0:

                control_dict = {'battery_charge': p_char, # vteni batareas zedmeti mzis energiit 
                                    'battery_discharge': 0, 
                                    'grid_import': 0,
                                    'grid_export': max(0,pv-load-p_char),#abs(min(load-pv,0)),
                                    'pv_consummed': min(pv, load+p_char),
                                    'genset': 0

                               }
            
            load_per_day[int(i/24)] += load
            
        else: # next days
            if k:
                day1 = load_per_day.index(min(load_per_day))
                load_per_day.remove(load_per_day[day1])
                day2 = load_per_day.index(min(load_per_day))
                k = False
                if abs(day1-day2) > 1:
                    friday = 5
                elif day1 == 0 or day2 == 0: 
                    friday = 6 
                else:
                    friday = min(day1,day2)-1 
#             print(f"Load: {load} PV: {pv} Price: {grid_price_import} C_charge {capa_to_charge} C_disc {capa_to_dischare}")
            if (i/24)%7 != day1 or (i/24)%7 != day2:
                if grid_price_import == min(price_seen) and pv < load:
                    control_dict = {'battery_charge': 0,
                                        'battery_discharge': p_disc/2,
                                        'grid_import': max(0, load-pv - p_disc/2),
                                        'grid_export':0,
                                        'pv_consummed': min(pv, load),
                                   }
                elif grid_price_import != min(price_seen) and pv < load:
                    control_dict = {'battery_charge': 0,
                                        'battery_discharge': p_disc,
                                        'grid_import': max(0, load-pv-p_disc),
                                        'grid_export':0,
                                        'pv_consummed': min(pv, load),
                                   }
                elif pv > load:
                    control_dict = {'battery_charge': p_char,
                                        'battery_discharge': 0,
                                        'grid_import': 0,
                                        'grid_export':max(0,pv-load-p_char),
                                        'pv_consummed': min(pv, load),
                                   }
            else:
                if grid_price_import == min(price_seen) and pv < load:
                    control_dict = {'battery_charge': 0,
                                        'battery_discharge': p_disc/3,
                                        'grid_import': max(0, load-pv-p_disc/3),
                                        'grid_export':0,
                                        'pv_consummed': min(pv, load),
                                   }
                elif grid_price_import != min(price_seen) and pv < load:
                    control_dict = {'battery_charge': 0,
                                        'battery_discharge': p_disc,
                                        'grid_import': max(0, load-pv-p_disc),
                                        'grid_export':0,
                                        'pv_consummed': min(pv, load),
                                   }
                elif pv > load:
                    control_dict = {'battery_charge': p_char,
                                        'battery_discharge': 0,
                                        'grid_import': 0,
                                        'grid_export':max(0,pv-load-p_char),
                                        'pv_consummed': min(pv, load),
                                   }
#             print(control_dict)    
        i+=1
        building_data = building.run(control_dict)
        total_building_cost += building.get_cost()
    return total_building_cost

In [ ]:
"""
Train code
"""

train_start = time.process_time()

model.learn(total_timesteps=365*24)

train_end = time.process_time()

train_frugality = train_end - train_start


In [ ]:
"""
Below is our model that uses rule based and deep RL hybrid approach
  
"""
building_env = MicroGridEnv(env_config={'microgrid':building_3, 'testing': True})

test_start = time.process_time()
total_building_costs = [0,0,0]

for i, building in enumerate(buildings[0:2]):

    building.reset(testing = True)

    total_building_cost = rule_based(building)
    total_building_costs[i] = total_building_cost
    
obs = building_env.reset(testing=True)
done = False
while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = building_env.step(action)

    total_building_costs[2]+=reward

test_end = time.process_time()

test_frugality = test_end - test_start

for i in range(99):
    obs = building_env.reset(testing=True)
    done = False
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = building_env.step(action)

        total_building_costs[2]+=reward
total_building_costs[2] = total_building_costs[2]/100

In [ ]:
final_results = {
    "building_1_performance" : total_building_costs[0],
    "building_2_performance" : total_building_costs[1],
    "building_3_performance" : -1 * total_building_costs[2],
    "frugality" : train_frugality + test_frugality,
}
print(final_results)